In [1]:
import tensorflow_datasets as tf
from tensorflow.keras.utils import to_categorical
import tensorflow as t;

In [2]:
(train_ds, train_labels), (test_ds, test_labels) = tf.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

In [3]:
train_ds = t.image.resize(train_ds, (150, 150))
test_ds = t.image.resize(test_ds, (150, 150))

In [4]:
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [5]:

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [6]:
#load vgg16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

In [7]:
#preprocessinpu
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [8]:
#to remove classification layer set it to false
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [9]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

In [10]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/10
65/65 [==============================] - 233s 4s/step - loss: 1.9325 - accuracy: 0.4190 - val_loss: 1.2165 - val_accuracy: 0.5214
Epoch 2/10
65/65 [==============================] - 230s 4s/step - loss: 0.8790 - accuracy: 0.6603 - val_loss: 1.1124 - val_accuracy: 0.6420
Epoch 3/10
65/65 [==============================] - 248s 4s/step - loss: 0.6356 - accuracy: 0.7645 - val_loss: 0.9331 - val_accuracy: 0.6770
Epoch 4/10
65/65 [==============================] - 224s 3s/step - loss: 0.4865 - accuracy: 0.8185 - val_loss: 1.1174 - val_accuracy: 0.6615
Epoch 5/10
65/65 [==============================] - 208s 3s/step - loss: 0.3966 - accuracy: 0.8628 - val_loss: 1.1142 - val_accuracy: 0.6926
Epoch 6/10
57/65 [=========================>....] - ETA: 19s - loss: 0.2697 - accuracy: 0.9035